In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("CyberSecurityThreatDetection").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/26 12:14:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/26 12:14:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.under_sampling import RandomUnderSampler

In [5]:
df = pd.read_csv('/home/CholePuri/Downloads/combined_dataset.csv')

In [6]:
# --- Data Cleaning and Preprocessing ---

# Handle missing values (example: fill with 0 or mean, depending on the column)
# Filling numerical NaNs with 0 and categorical with mode
for col in df.columns:
    if df[col].dtype in ['int64', 'float64']:
        df[col].fillna(0, inplace=True)
    else:
        df[col].fillna(df[col].mode()[0], inplace=True)

# Drop irrelevant or unnecessary columns
columns_to_drop = [
    'Flow ID', 'Source IP', 'Destination IP', 'Source Port', 'Destination Port',
    'Protocol', 'Timestamp' # We will process Timestamp separately if needed for features, but drop the raw column
]
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns], errors='ignore')

# Handle timestamp column - create relevant features
if 'Timestamp' in pd.read_csv('/home/CholePuri/Downloads/combined_dataset.csv').columns: # Check if it exists in the original file
    original_df = pd.read_csv('/home/CholePuri/Downloads/combined_dataset.csv')
    df['Timestamp'] = pd.to_datetime(original_df['Timestamp'])
    df['hour_of_day'] = df['Timestamp'].dt.hour
    df['day_of_week'] = df['Timestamp'].dt.dayofweek
    # You can add more time-based features like minute, second, etc.
    df = df.drop(columns=['Timestamp'], errors='ignore') # Drop after creating features

# Encode categorical features
for col in df.select_dtypes(include='object').columns:
    if col != 'Label': # Don't encode the target variable yet
        df[col] = LabelEncoder().fit_transform(df[col])

/tmp/ipykernel_12390/1985714857.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(0, inplace=True)
/tmp/ipykernel_12390/1985714857.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.metho

In [7]:
df.head()

,Dst Port,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,...,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Src IP,Src Port,Dst IP,hour_of_day,day_of_week
0,80,10278,2,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,DDOS attack-HOIC,1,0.0,0,2,2
1,22,18,1,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,SSH-Bruteforce,1,0.0,0,3,2
2,22,6,1,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,SSH-Bruteforce,1,0.0,0,2,2
3,53,101638,1,1,40.0,146.0,40.0,40.0,40.0,0.0,...,0.0,0.0,0.0,0.0,Infilteration,1,0.0,0,11,2
4,80,119927451,119775,0,3832800.0,0.0,32.0,32.0,32.0,0.0,...,0.0,0.0,0.0,0.0,DDOS attack-LOIC-UDP,1,0.0,0,10,2


In [8]:
df.to_csv("preprocessed_dataset.csv", index=False)